In [ ]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from phoenix.server.api.types.pagination import (
    Cursor,
    CursorSortColumn,
    CursorSortColumnDataType,
)

project_id = "UHJvamVjdDox"
client = Client(
    transport=RequestsHTTPTransport(url="http://127.0.0.1:6006/graphql", timeout=100),
    fetch_schema_from_transport=True,
)

In [ ]:
cursor = "MTAwOkRBVEVUSU1FOjIwMjMtMTItMTFUMTc6NDQ6MDIuNTM0MTI5KzAwOjAw"
cursor = Cursor.from_string(cursor)
print(cursor)

In [ ]:
# test query for doing sanity checks
response = client.execute(
    gql(
        """query SpansTableSpansQuery($after: String = null, $filterCondition: String = null, $first: Int = 100, $sort: SpanSort = {col: startTime, dir: desc}, $timeRange: TimeRange, $id: GlobalID!) {
  node(id: $id) {
    ... on Project {
      spans(
        first: $first
        after: $after
        sort: $sort
        filterCondition: $filterCondition
        timeRange: $timeRange
      ) {
        edges {
          cursor
        }
        pageInfo {
          endCursor
          hasNextPage
        }
      }
    }
  }
}"""
    ),
    variable_values={
        "after": None,
        "filterCondition": "",
        "first": 100,
        "sort": {"col": "startTime", "dir": "desc"},
        "timeRange": {
            "start": "2024-04-28T17:00:00.000Z",
            "end": "2025-05-05T17:00:00.000Z",
        },
        "id": "UHJvamVjdDox",
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
ids

In [ ]:
spans_query = gql(
    """query SpansQuery($projectId: GlobalID!, $after: String = null, $before: String = null, $filterCondition: String = null, $first: Int = null, $last: Int = null, $sort: SpanSort = null) {
  node(id: $projectId) {
    ... on Project {
      spans(
        after: $after
        before: $before
        filterCondition: $filterCondition
        first: $first
        last: $last
        rootSpansOnly: false
        sort: $sort
      ) {
        edges {
          cursor
        }
        pageInfo {
          hasNextPage
          hasPreviousPage
          startCursor
          endCursor
        }
      }
    }
  }
}"""
)

In [ ]:
# basic query
response = client.execute(
    spans_query,
    variable_values={"projectId": project_id, "first": 5},
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [1, 2, 3, 4, 5], ids

In [ ]:
# query with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": str(Cursor(rowid=755)),
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [756, 757, 758, 759, 760], ids

In [ ]:
# page ends on the penultimate record and excludes last record
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": str(Cursor(759)),
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert ids == [760, 761, 762, 763, 764], ids
assert has_next_page is True
assert has_previous_page is False

In [ ]:
# page ends on the last record exactly
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": str(Cursor(760)),
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert ids == [761, 762, 763, 764, 765], ids
assert has_next_page is False
assert has_previous_page is False

In [ ]:
# page ends before it reaches the limit
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": str(Cursor(761)),
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert ids == [762, 763, 764, 765], ids
assert has_next_page is False
assert has_previous_page is False

In [ ]:
# basic filter condition
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "first": 5,
        "filterCondition": "span_kind == 'LLM'",
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    5,
    10,
    15,
    20,
    25,
], ids

In [ ]:
# basic filter condition with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "first": 5,
        "after": str(Cursor(5)),  # skip the first span satisfying the filter condition
        "filterCondition": "span_kind == 'LLM'",
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    10,
    15,
    20,
    25,
    30,
], ids

In [ ]:
# compound filter condition with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": str(Cursor(10)),  # skip the first span satisfying the filter condition
        "first": 5,
        "filterCondition": "span_kind == 'LLM' and cumulative_llm_token_count_prompt > 300",
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    15,
    30,
    35,
    45,
    60,
], ids

In [ ]:
# order by descending start time
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "desc"},
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
end_cursor = Cursor.from_string(end_cursor)
assert ids == [765, 764, 763, 762, 761], ids
assert end_cursor.rowid == 761
assert (end_sort_column := end_cursor.sort_column) is not None
assert (
    end_node_start_timestamp := end_sort_column.value.isoformat()
) == "2023-12-11T17:48:40.807667+00:00", end_node_start_timestamp

In [ ]:
# order by ascending start time
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "asc"},
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
end_cursor = Cursor.from_string(end_cursor)
assert ids == [1, 2, 3, 4, 5], ids
assert end_cursor.rowid == 5
assert (end_sort_column := end_cursor.sort_column) is not None
assert (
    end_node_start_timestamp := end_sort_column.value.isoformat()
) == "2023-12-11T17:43:23.712144+00:00", end_node_start_timestamp

In [ ]:
# order by cumulative prompt token count in descending order
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "cumulativeTokenCountPrompt", "dir": "desc"},
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    710,
    709,
    706,
    645,
    644,
], ids

In [ ]:
# order by cumulative prompt token count in ascending order
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "cumulativeTokenCountPrompt", "dir": "asc"},
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    2,
    3,
    7,
    8,
    12,
], ids

In [ ]:
# order by descending start time with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "desc"},
        "first": 5,
        "after": str(
            Cursor(
                758,
                sort_column=CursorSortColumn.from_string(
                    type=CursorSortColumnDataType.DATETIME,
                    cursor_string="2023-12-11T17:48:39.949837+00:00",
                ),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
start_cursor = response["node"]["spans"]["pageInfo"]["startCursor"]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
start_cursor = Cursor.from_string(start_cursor)
end_cursor = Cursor.from_string(end_cursor)
assert ids == [
    757,
    756,
    755,
    754,
    753,
], ids
assert (start_sort_column := start_cursor.sort_column) is not None
assert (
    start_timestamp := str(start_sort_column)
) == "2023-12-11T17:48:39.949695+00:00", start_timestamp
assert (
    start_field_type := start_sort_column.type
) == CursorSortColumnDataType.DATETIME, start_field_type
assert (end_sort_column := end_cursor.sort_column) is not None
assert (end_timestamp := str(end_sort_column)) == "2023-12-11T17:48:38.603846+00:00", end_timestamp
assert (end_field_type := end_sort_column.type) == CursorSortColumnDataType.DATETIME, end_field_type

In [ ]:
# order by ascending start time with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "asc"},
        "first": 5,
        "after": str(
            Cursor(
                9,
                sort_column=CursorSortColumn.from_string(
                    type=CursorSortColumnDataType.DATETIME,
                    cursor_string="2023-12-11T17:43:25.842986+00:00",
                ),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
start_cursor = response["node"]["spans"]["pageInfo"]["startCursor"]
end_cursor = response["node"]["spans"]["pageInfo"]["endCursor"]
start_cursor = Cursor.from_string(start_cursor)
end_cursor = Cursor.from_string(end_cursor)
assert ids == [
    10,
    11,
    12,
    13,
    14,
], ids
assert (start_sort_column := start_cursor.sort_column) is not None
assert (
    start_timestamp := str(start_sort_column)
) == "2023-12-11T17:43:25.844758+00:00", start_timestamp
assert (
    start_field_type := start_sort_column.type
) == CursorSortColumnDataType.DATETIME, start_field_type
assert (end_sort_column := end_cursor.sort_column) is not None
assert (end_timestamp := str(end_sort_column)) == "2023-12-11T17:43:26.704532+00:00", end_timestamp
assert (end_field_type := end_sort_column.type) == CursorSortColumnDataType.DATETIME, end_field_type

In [ ]:
# order by cumulative prompt token count in descending order with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "cumulativeTokenCountPrompt", "dir": "desc"},
        "first": 5,
        "after": str(
            Cursor(
                rowid=644,  # row 644 is in between rows 645 and 641, which also have 1054 cumulative prompt tokens
                sort_column=CursorSortColumn(type=CursorSortColumnDataType.FLOAT, value=1054),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    641,
    550,
    549,
    546,
    60,
], ids

In [ ]:
# order by cumulative prompt token count in ascending order with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "cumulativeTokenCountPrompt", "dir": "asc"},
        "first": 5,
        "after": str(
            Cursor(
                rowid=294,  # row 294 is in between rows 295 and 291, which also have 276 cumulative prompt tokens
                sort_column=CursorSortColumn(type=CursorSortColumnDataType.INT, value=276),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    295,
    111,
    114,
    115,
    21,
], ids

In [ ]:
# order by hallucination eval label in descending order
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {
            "evalResultKey": {"name": "Hallucination", "attr": "label"},
            "dir": "desc",
        },
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [761, 756, 746, 741, 721], ids

In [ ]:
# order by hallucination eval label in ascending order
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {
            "evalResultKey": {"name": "Hallucination", "attr": "label"},
            "dir": "asc",
        },
        "first": 5,
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [6, 21, 26, 31, 41], ids

In [ ]:
# order by hallucination eval label in descending order with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {
            "evalResultKey": {"name": "Hallucination", "attr": "label"},
            "dir": "desc",
        },
        "first": 5,
        "after": str(
            Cursor(
                rowid=16,  # row 141 is surrounded by many other hallucinations
                sort_column=CursorSortColumn(
                    type=CursorSortColumnDataType.STRING, value="hallucinated"
                ),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    11,
    1,
    751,
    736,
    731,
], ids

In [ ]:
# order by hallucination eval label in ascending order with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {
            "evalResultKey": {"name": "Hallucination", "attr": "label"},
            "dir": "asc",
        },
        "first": 5,
        "after": str(
            Cursor(
                rowid=731,  # row 746 is surrounded by many other hallucinations
                sort_column=CursorSortColumn(type=CursorSortColumnDataType.STRING, value="factual"),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    736,
    751,
    1,
    11,
    16,
], ids

In [ ]:
# order by hallucination eval score in descending order with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {
            "evalResultKey": {"name": "Hallucination", "attr": "score"},
            "dir": "desc",
        },
        "first": 5,
        "after": str(
            Cursor(
                rowid=21,
                sort_column=CursorSortColumn(type=CursorSortColumnDataType.FLOAT, value=1),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    6,
    761,
    756,
    746,
    741,
], ids

In [ ]:
# order by hallucination eval score in ascending order with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {
            "evalResultKey": {"name": "Hallucination", "attr": "score"},
            "dir": "asc",
        },
        "first": 5,
        "after": str(
            Cursor(
                rowid=746,
                sort_column=CursorSortColumn(type=CursorSortColumnDataType.FLOAT, value=0),
            )
        ),
    },
)
cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
ids = [Cursor.from_string(cursor).rowid for cursor in cursors]
assert ids == [
    756,
    761,
    6,
    21,
    26,
], ids